In [1]:
import os
import pandas as pd
import re
from sqlalchemy import create_engine

In [2]:
xls_sku_dtype = {"skuId":"int64"
    ,"sku名称":"object"
    ,"支付金额":"object"
    ,"支付买家数":"object"
    ,"支付件数":"object"
    ,"加购件数":"object"
                            }

In [3]:
def df_to_sql(df_to_dwh,dwh_list):
    username = "root"
    password = "9518llhh731X"
    host = "localhost"
    port = 3306
    database = "dwh"
    charset = "utf8mb4"
    connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}?charset={charset}'
    conn = create_engine(connection_string)
    pd.io.sql.to_sql(df_to_dwh,dwh_list,con=conn,if_exists="append",index=False)
    return

In [4]:
skuList = ["塞德思","希纳露","百热朗","健眠计划","棒维素","琵力消","整肠药"]
def sku_tag_add(itemName,itemLists):
    for item in itemLists:
        if item in itemName:
            return item
        else:
            continue
    return '灵的融'

In [5]:
def get_sku_sales_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    sku_date = re.findall(date_pattern,xls_file_name)[0]
    return sku_date

In [6]:
sku_sales_xls_list=os.listdir(r"D:\盐野义\数据\sku_data")

In [7]:
sku_data = pd.read_excel(io="D:/盐野义/数据/sku_data/"+sku_sales_xls_list[0],skiprows=5,engine="xlrd",dtype=xls_sku_dtype)
sku_data["日期"] = pd.to_datetime(get_sku_sales_date(sku_sales_xls_list[0]))

In [8]:
for xls_file in sku_sales_xls_list[1:]:
    sku_data_sub = pd.read_excel(io="D:/盐野义/数据/sku_data/"+xls_file,skiprows=5,engine="xlrd",dtype=xls_sku_dtype)
    sku_data_sub["日期"] = pd.to_datetime(get_sku_sales_date(xls_file))
    sku_data = pd.concat([sku_data,sku_data_sub],axis=0,ignore_index=True)

In [9]:
sku_data["商品别名"] = sku_data["sku名称"].apply(lambda x:sku_tag_add(x,itemLists=skuList))


In [10]:
sku_data["支付金额"] = sku_data["支付金额"].apply(lambda x:x.replace(",",""))
sku_data["支付买家数"] = sku_data["支付买家数"].apply(lambda x:x.replace(",",""))
sku_data["支付件数"] = sku_data["支付件数"].apply(lambda x:x.replace(",",""))
sku_data["加购件数"] = sku_data["加购件数"].apply(lambda x:x.replace(",",""))

In [11]:
sku_data = sku_data.astype({"日期":"datetime64[ns]"
    ,"skuId":"int64"
    ,"sku名称":"object"
    ,"支付金额":"float64"
    ,"支付买家数":"int64"
    ,"支付件数":"int64"
    ,"加购件数":"int64"
                            })


In [12]:
df_to_sql(df_to_dwh=sku_data,dwh_list="yyy_sku")